In [0]:
!wget -q https://www.dropbox.com/s/6s06zacfu9jn4z8/Patrick%20Jean-Baptiste%20-%20Emotion_Image_Data_Modified.zip?dl=1 -O Emotion_Images.zip

!wget -q https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Song_Actors_01-24.zip
!wget -q https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Speech_Actors_01-24.zip

!unzip -q -d ./song Audio_Song_Actors_01-24.zip 
!unzip -q -d ./speech Audio_Speech_Actors_01-24.zip
!unzip -q Emotion_Images.zip

!wget -q https://www.dropbox.com/s/yigijs122togfk4/embeddings.dat?dl=1  -O embeddings.dat
!wget -q https://www.dropbox.com/s/qdhtexle4p0ngc3/DatasetSplitCSV.zip?dl=1 -O DatasetSplitCSV.zip
!unzip -q DatasetSplitCSV.zip


In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import glob

# example of converting an image with the Keras API
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img


Using TensorFlow backend.


In [0]:
def mapReduce(embed,funcName):
    if funcName=="Avg":
        embed= [np.average(embed,axis=0)]
    if funcName=="Pad":
        embed=np.pad(embed, [( 0,6-embed.shape[0]), (0, 0)], mode='constant', constant_values=0)
        embed= [embed.reshape(-1)]
    if funcName=="Many2One":
        embed=[embed[i,:] for i in range(embed.shape[0])]
    if funcName=="None":
        embed=[embed]
    return embed

def uint8_to_float32(x):
    return (np.float32(x) - 128.) / 128.

In [0]:
#Parameters
params={}
params["embeddingType"]="post" # post,raw
params["mapReduceFunc"]="Pad" # Avg,Pad,Many2One

In [0]:
train=pd.read_csv("train.csv",header=None)
valid=pd.read_csv("valid.csv",header=None)
test=pd.read_csv("test.csv",header=None)

embeddings=np.load("embeddings.dat",allow_pickle=True)
# embeddings["post"]['03-02-04-01-02-02-02.wav'].dtype
seconds={}

for df in [train,test,valid]:
    for i in df.index:
        df.at[i, 0] = df.iloc[i][0][3:]

embeddings2={"post":{}}
for key,embed in embeddings["post"].items():
    embeddings2["post"][key[3:]]=embed
embeddings=embeddings2

for key,embed in embeddings["post"].items():
    embeddings["post"][key]=uint8_to_float32(embed)
    seconds[key]=embed.shape[0]


In [0]:
x_train=[]
x_val=[]
x_test=[]
for x in train[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_train.extend(mapReduce(embed,params["mapReduceFunc"]))

for x in valid[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_val.extend(mapReduce(embed,params["mapReduceFunc"]))

for x in test[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_test.extend(mapReduce(embed,params["mapReduceFunc"]))

x_train=np.array(x_train)
x_val=np.array(x_val)
x_test=np.array(x_test)

y_train=np.array(train[1].astype('category').cat.codes)
y_val=np.array(valid[1].astype('category').cat.codes)
y_test=np.array(test[1].astype('category').cat.codes)

if params["mapReduceFunc"]=="Many2One":
    # print("a")
    y_train2,y_val2,y_test2=[],[],[]
    for x,y,y2 in zip([train[0],valid[0],test[0]],[y_train,y_val,y_test],[y_train2,y_val2,y_test2]):
        for i,embedName in enumerate(x):
            # print(seconds[embedName])
            y2.extend([y[i] for k in range(seconds[embedName])])
        # print(y2)
    y_train,y_val,y_test=[np.array(y) for y in [y_train2,y_val2,y_test2]]


In [96]:
len(imageList),len(x_train)+len(x_val)+len(x_test)

(4902, 2452)

In [0]:
imageList=glob.glob("Emotion_Image_Data_Modified/Emotion_Images/*")
imageDict={}
for img in imageList:
    fname=(img).split("/")[-1].split(".")[0]+".wav"
    fname=fname[3:]
    imageDict.setdefault(fname,[])
    imageDict[fname].append(img)

for k,v in imageDict.items():
    if len(v)<2:
        imageDict[k].append(v[0])
        # print(k)
    v[0] = img_to_array(load_img(v[0]))
    v[1] = img_to_array(load_img(v[1]))
    # imageDict[k]=np.concatenate(v,axis=0)
    imageDict[k]=v[0]

In [0]:


x_trainImg=[]
x_valImg=[]
x_testImg=[]
for x in train[0]:
    image=[imageDict[k]]
    x_trainImg.extend(image)


for x in valid[0]:
    image=[imageDict[k]]
    x_valImg.extend(image)

for x in test[0]:
    image=[imageDict[k]]
    x_testImg.extend(image)

x_trainImg=np.array(x_trainImg)
x_valImg=np.array(x_valImg)
x_testImg=np.array(x_testImg)

In [0]:
# from __future__ import print_function
# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K


# batch_size = 128
# num_classes = 10
# epochs = 12

# input image dimensions
# img_rows, img_cols = 256, 128

# the data, split between train and test sets

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [25]:
x_train.shape

(60000, 28, 28)

In [26]:
# inputs = keras.Input(shape=(x_train.shape[-1],), name="log-mel")
# x = tf.keras.layers.Reshape((1, 128), input_shape=(x_train.shape[-1],))(inputs)
# x = layers.LSTM(128,input_shape=((1,128)))(x)
# # x = layers.Dense(128, activation="tanh", name="dense_1")(inputs)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(64, activation="tanh", name="dense_2")(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(32, activation="tanh", name="dense_3")(x)
# x = layers.Dropout(0.5)(x)
# outputs = layers.Dense(8, activation="softmax", name="predictions")(x)

# model = keras.Model(inputs=inputs, outputs=outputs)

'channels_last'

In [44]:
x_train.shape

(1470, 768)

In [0]:
inputs = keras.Input(shape=(x_train.shape[-1],), name="log-mel")
# x = tf.keras.layers.Reshape((1, 128), input_shape=(x_train.shape[-1],))(inputs)
# x = layers.LSTM(128,input_shape=((1,128)))(x)
x = layers.Dense(128, activation="tanh", name="dense_1")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="tanh", name="dense_2")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32, activation="tanh", name="dense_3")(x)
x = layers.Dropout(0.5)(x)
outputs2 = layers.Dense(8, activation="relu")(x)

model2 = keras.Model(inputs=inputs, outputs=outputs2)

In [0]:
VInputs = keras.Input(shape=(256,128,3), name="images")
v=layers.Conv2D(32, 3,activation='relu' )(VInputs)
v=layers.Conv2D(64, 3, activation='relu')(v)
v=layers.MaxPooling2D(pool_size=(2, 2))(v)
v=layers.Dropout(0.25)(v)
v=layers.Flatten()(v)
v=layers.Dense(128, activation='relu')(v)
v=layers.Dense(8, activation='relu')(v)
output=layers.Dropout(0.25)(v)

model1 = keras.Model(inputs=VInputs, outputs=output)

In [0]:
mergedOut = layers.Add()([model1.output,model2.output])
mergedOut=layers.Dense(8, activation='softmax')(mergedOut)


newModel = keras.Model([model1.input,model2.input], mergedOut)
newModel.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer='adam',
                metrics=['accuracy'])

In [0]:
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                               factor=0.2,
                               patience=5,
                               min_lr=1e-6,
                               verbose=1)

In [0]:
my_callbacks=[lr_reducer,
              tf.keras.callbacks.EarlyStopping(patience=50),
              tf.keras.callbacks.TensorBoard(log_dir='./logs')]

   

In [109]:
newModel.summary()

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
images (InputLayer)             [(None, 256, 128, 3) 0                                            
__________________________________________________________________________________________________
log-mel (InputLayer)            [(None, 768)]        0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 254, 126, 32) 896         images[0][0]                     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          98432       log-mel[0][0]                    
___________________________________________________________________________________________